In [1]:
import json

from sklearn.model_selection import train_test_split
import pandas as pd
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import math

from torch.utils.data import Dataset, DataLoader

## Load Data

In [34]:
#data set from https://data.world/datafiniti/pizza-restaurants-and-pizzas-on-their-menus

In [106]:
data = pd.read_csv('pizza_sales/full.csv')#changed the name of the file

In [26]:
data_extracted = data[['id','dateAdded','address','menus.name','name']]

In [27]:
data_extracted

,id,dateAdded,address,menus.name,name
0,AVz3Y-7h3D1zeR_xDAqm,2017-06-30T05:05:40Z,4203 E Kiehl Ave,Cheese Pizza,Shotgun Dans Pizza
1,AVweGPFF_7pvs4fzAAzQ,2016-04-02T04:02:49Z,25 E Camelback Rd,Pizza Cookie,Sauce Pizza Wine
2,AVwdRGa9_7pvs4fz4E3K,2016-03-03T18:39:49Z,3703 Paxton Ave,Pizza Blanca,Mios Pizzeria
3,AVwdX4psIN2L1WUfvJB1,2016-03-29T05:08:59Z,30495 John R Rd,Small Pizza,Hungry Howies Pizza
4,AVwdaeTtkufWRAb55pSH,2016-03-31T02:34:04Z,3600 Eastern Ave,Pizza Sub,Spartan Pizzeria
...,...,...,...,...,...
9995,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,1126 E Lovejoy St,Super Steak Pizza,Carbone's Pizzeria
9996,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,1126 E Lovejoy St,Large Cheese & 1 Item Pizza,Carbone's Pizzeria
9997,AVz6kzsQFcQ3k02bDhwQ,2017-06-30T19:56:11Z,3641 E Main St,"Individual 6"" Pepperoni Stuffed Deep Dish Pizza",Giordano's Pizza
9998,AVz6kzsQFcQ3k02bDhwQ,2017-06-30T19:56:11Z,3641 E Main St,"Individual 6"" Cheese Stuffed Deep Dish Pizza",Giordano's Pizza


In [44]:
has_next_dish = data_extracted.groupby('id').count()[data_extracted.groupby('id').count()['name']>2].index

In [47]:
filtered_data = data_extracted[data_extracted['id'].isin(has_next_dish)]
filtered_data

,id,dateAdded,address,menus.name,name
10,AVwchRVZkufWRAb5wDIV,2016-03-29T04:47:31Z,4061 28th St SE,Kids Cheese Pizza,Carrabbas Italian Grill
11,AVwchRVZkufWRAb5wDIV,2016-03-29T04:47:31Z,4061 28th St SE,Create Your Own Pizza,Carrabbas Italian Grill
12,AVwchRVZkufWRAb5wDIV,2016-03-29T04:47:31Z,4061 28th St SE,Prosciutto Arugula Pizza,Carrabbas Italian Grill
13,AVwchRVZkufWRAb5wDIV,2016-03-29T04:47:31Z,4061 28th St SE,Kids Pepperoni Pizza,Carrabbas Italian Grill
14,AVwc7s1wIN2L1WUfqehD,2015-10-21T17:51:11Z,146 N Glendora Ave,Three Cheese Pizza,Domenicos Jr
...,...,...,...,...,...
9992,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,1126 E Lovejoy St,White Pizza,Carbone's Pizzeria
9993,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,1126 E Lovejoy St,Cheese Burger Pizza,Carbone's Pizzeria
9994,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,1126 E Lovejoy St,Pizza Sub,Carbone's Pizzeria
9995,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,1126 E Lovejoy St,Super Steak Pizza,Carbone's Pizzeria


In [63]:
first_dish = filtered_data.groupby('id').nth(0)['menus.name'].tolist()
next_dish = filtered_data.groupby('id').nth(1)['menus.name'].tolist()

In [64]:
X = first_dish
y = next_dish

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = pd.DataFrame(data=X_train, columns=['prev_dish'])
X_test = pd.DataFrame(data=X_test, columns=['prev_dish'])
y_train = pd.DataFrame(data=y_train, columns=['next_dish'])
y_test = pd.DataFrame(data=y_test, columns=['next_dish'])

In [79]:
def encode_all(X_train, X_test, y_train, y_test):
    """
    Numeric Encoding of IDs.
    Function deals with unkown values in test by encoding them as zero.
    Inefficient, but gets the job done for now
    """
    # Thought itd be better to not have y encoding overlap with x (x has a 1, and y has 1, 2 etc)
    num_xdishes = len(X_train.prev_dish.unique())  # Variable for stopping y and x encoding from overlapping
    X_trainMap = {i+1:o for i,o in enumerate(X_train.prev_dish)}  # Create a map for encoding train data and checking test data
    X_train= pd.DataFrame(data=range(1, num_xdishes+1), columns=['prev_dish'])
    X_trainMap[0] = 'UNSEEN'

    for i in range(len(X_test)):
        if X_test.iloc[i][0] not in X_trainMap.values():  # If unseen, map to zero
            X_test.iloc[i][0] = 0
        else:
            X_test.iloc[i][0] = X_trainMap[X_test[i]]
    y_trainMap = {i+1:o for i,o in enumerate(y_train.next_dish)}
    y_train = pd.DataFrame(data=range(num_xSongs, num_xSongs + len(y_train)+1), columns=['next_dish'])
    y_trainMap[0] = 'UNSEEN'
    for i in range(len(y_test)):
        if y_test.iloc[i][0] not in y_trainMap.values():
            y_test.iloc[i][0] = 0
        else:
            y_test.iloc[i][0] = y_trainMap[y_test[i]]

    return X_train, X_test, y_train, y_test

In [89]:
def encode_all(X_train, X_test, y_train, y_test):
    # Unique dishes mapping to indices starting from 1
    unique_prev_dishes = pd.Series(X_train.prev_dish.unique()).dropna().reset_index(drop=True)
    unique_next_dishes = pd.Series(y_train.next_dish.unique()).dropna().reset_index(drop=True)
    
    # Create maps for encoding
    X_train_map = {dish: i+1 for i, dish in enumerate(unique_prev_dishes)}
    X_train_map['UNSEEN'] = 0
    y_train_map = {dish: i+1+len(unique_prev_dishes) for i, dish in enumerate(unique_next_dishes)}
    y_train_map['UNSEEN'] = 0
    
    # Apply encoding
    X_train['prev_dish'] = X_train['prev_dish'].map(X_train_map).fillna(0).astype(int)
    X_test['prev_dish'] = X_test['prev_dish'].map(X_train_map).fillna(0).astype(int)
    y_train['next_dish'] = y_train['next_dish'].map(y_train_map).fillna(0).astype(int)
    y_test['next_dish'] = y_test['next_dish'].map(y_train_map).fillna(0).astype(int)

    return X_train, X_test, y_train, y_test

In [88]:
X_trainMap = {i+1:o for i,o in enumerate(X_train.prev_dish)}
X_trainMap

{1: 'Pizza By the Slice',
 2: '1 Medium 1ampcomma Topping Pizza',
 3: 'Specialty Pizza',
 4: 'Putanesca Pizza',
 5: 'NY Style 1 Topping Pizza Slice',
 6: 'Cheese Burger Pizza',
 7: 'Chicken Pizza Steak',
 8: 'Veggie Combo Pizza',
 9: 'Stuffed Pizza',
 10: 'Tijuana Special Pizza',
 11: 'Pepper Steak Pizza',
 12: 'Taco Pizza',
 13: 'Ny Cheese Pizza',
 14: 'White Broccoli Pizza',
 15: 'Pizza By The Slice',
 16: 'Hand Tossed Pizza',
 17: 'Ziti Pizza',
 18: 'Pepperoni Pizza',
 19: 'For Stuffed Pizza',
 20: '14ampquot Deluxe Sicilian Pizza',
 21: '2 Medium Pizzas',
 22: 'Stuffed Veggie Pizza',
 23: 'Pizza Sub',
 24: 'Pizza Blanca',
 25: 'Kids Pizza',
 26: 'Combo Pizza',
 27: 'Hawaiian Pizza Baking Required',
 28: 'Mini Pizza',
 29: 'Chicken Bacon Ranch Pizza',
 30: 'Shrimp Scampi Pizza',
 31: 'Whole Wheat Vegan Pizza',
 32: 'Pizza Sub',
 33: 'Pepperoni Pizza Rolls',
 34: 'Bbq Chicken Pizza',
 35: 'Greek Pepperoni Pizza Baking Required',
 36: 'Pizza Romeo',
 37: 'Hawaiian Pizza',
 38: '10" In

In [83]:
X_test

,prev_dish
0,0
1,0
2,0
3,0
4,Bbq Chicken Pizza
...,...
211,Vegetable Pizza Primavera
212,Pepperoni Pizza Slice
213,Deluxe Pizza
214,Sicilian Pizza Slice


In [90]:
X_train, X_test, y_train, y_test = encode_all(X_train, X_test, y_train, y_test)
trainComb = pd.concat([X_train, y_train], axis=1)
testComb = pd.concat([X_test, y_test], axis=1)

In [91]:
trainComb["prev_dish"].fillna(28, inplace=True)
trainComb.sample(3)

,prev_dish,next_dish
445,348,947
821,580,705
577,36,845


In [92]:
trainComb["prev_dish"] = trainComb["prev_dish"].astype(int)
testComb["prev_dish"] = testComb["prev_dish"].astype(int)
testComb["next_dish"] = testComb["next_dish"].astype(int)
trainComb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   prev_dish  860 non-null    int64
 1   next_dish  860 non-null    int64
dtypes: int64(2)
memory usage: 13.6 KB


## Pytorch Dataset

In [93]:
y_train.shape

(860, 1)

In [94]:
class OurData(Dataset):
    def __init__(self, x, y):
        # x = torch.from_numpy(x) #
        # y = torch.from_numpy(y)
        self.x, self.y = x, y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [95]:
xTrain_vals = trainComb["prev_dish"].values
yTrain_vals = trainComb["next_dish"].values
xTrain_vals = torch.Tensor(xTrain_vals).requires_grad_(True)
yTrain_vals = torch.Tensor(yTrain_vals).requires_grad_(True)

xTest_vals = testComb["prev_dish"].values
yTest_vals = testComb["next_dish"].values
xTest_vals = torch.Tensor(xTest_vals).requires_grad_(True)
yTest_vals = torch.Tensor(yTest_vals).requires_grad_(True)

In [96]:
train_ds = OurData(xTrain_vals, yTrain_vals)
valid_ds = OurData(xTest_vals, yTest_vals)

In [97]:
train_ds.x.shape, train_ds.y.shape

(torch.Size([860]), torch.Size([860]))

In [98]:
train_dl = DataLoader(train_ds, batch_size=7, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=7, shuffle=False)

## Modeling:

In [99]:
emb_size = 512
model = nn.Sequential(
        nn.Embedding(len(xTrain_vals)+1, emb_size),
        nn.ELU(),
        nn.Linear(emb_size, len(xTrain_vals)),  # output is a 28x28
        nn.Softmax(dim=1),
        )


In [100]:
def val_metric(model, valid_dl):
    model.eval()
    losses = []
    y_hats = []
    ys = []
    for x, y in valid_dl:  # Get batches from valid dl, but we use entire valdiation set
        y = y.unsqueeze(1)
        y_hat = model(x.long())
        y_hat = torch.argmax(y_hat, dim=1)
        loss = F.mse_loss(y_hat.float(), y.float())
        y_hats.append(y_hat.detach().numpy())
        ys.append(y.detach().numpy())
        losses.append(loss.item())

    ys = np.concatenate(ys)
    y_hats = np.concatenate(y_hats)
    return np.mean(losses)

In [101]:
## train_loop function
def train_loop(model, train_dl, valid_dl, optimizer, epochs):
    losses = []
    for i in range(epochs):
        model.train()
        for x, y in train_dl:  # Loop where X = 1 batch
            y = y.unsqueeze(1)
            y_hat = model(x.long())
            y_hat = torch.argmax(y_hat, dim=1)
            loss = F.mse_loss(y_hat.float(), y.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        train_loss = np.mean(losses)
        valid_loss = val_metric(model, valid_dl)  # Loops thru entire val set
        print(f"loss on epoch {i} = %.3f" %
              (train_loss))

In [102]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [103]:
train_loop(model, train_dl, valid_dl, optimizer, epochs=4)

/var/folders/qz/q2n3jxkd5857jpcny7twjyvr0000gn/T/ipykernel_16783/2358871109.py:10: UserWarning: Using a target size (torch.Size([7, 1])) that is different to the input size (torch.Size([7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat.float(), y.float())
/var/folders/qz/q2n3jxkd5857jpcny7twjyvr0000gn/T/ipykernel_16783/2358871109.py:10: UserWarning: Using a target size (torch.Size([6, 1])) that is different to the input size (torch.Size([6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat.float(), y.float())


loss on epoch 0 = 270855.134
loss on epoch 1 = 271300.217
loss on epoch 2 = 270841.943
loss on epoch 3 = 271176.161


In [104]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
train_loop(model, train_dl, valid_dl, optimizer, epochs=4)

/var/folders/qz/q2n3jxkd5857jpcny7twjyvr0000gn/T/ipykernel_16783/2358871109.py:10: UserWarning: Using a target size (torch.Size([7, 1])) that is different to the input size (torch.Size([7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat.float(), y.float())
/var/folders/qz/q2n3jxkd5857jpcny7twjyvr0000gn/T/ipykernel_16783/2358871109.py:10: UserWarning: Using a target size (torch.Size([6, 1])) that is different to the input size (torch.Size([6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat.float(), y.float())


loss on epoch 0 = 272102.960
loss on epoch 1 = 271232.295
loss on epoch 2 = 271043.449
loss on epoch 3 = 271158.904


In [105]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
train_loop(model, train_dl, valid_dl, optimizer, epochs=4)

/var/folders/qz/q2n3jxkd5857jpcny7twjyvr0000gn/T/ipykernel_16783/2358871109.py:10: UserWarning: Using a target size (torch.Size([7, 1])) that is different to the input size (torch.Size([7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat.float(), y.float())
/var/folders/qz/q2n3jxkd5857jpcny7twjyvr0000gn/T/ipykernel_16783/2358871109.py:10: UserWarning: Using a target size (torch.Size([6, 1])) that is different to the input size (torch.Size([6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat.float(), y.float())


loss on epoch 0 = 270422.372
loss on epoch 1 = 270648.202
loss on epoch 2 = 270694.259
loss on epoch 3 = 270865.671
